# Zad 1.  
Zbadamy ewolucję układu na sieci kwadratowej o rozmiarze 100 ×100 dla początkowego losowego rozkładu dwóch opinii w układzie. Sporządzamy również wykres liczby opinii na ‘tak’ od czasu. Przyjmijmy sekwencyjny schemat aktualizacji opinii.  

In [1]:
# packages required
using Random, DataFrames, CSV

In [2]:
L = 100;

In [3]:
function voter_model(; L=100, max_iter=100_000, verbose=true)

    network = rand(0:1, (L, L)) # L x L randomized binary matrix {1: "yes", 0: "no")
    i = 0                       # iteration counter
    yesses = []                 # number of ones/yes answers container

    # model loop
    start_time = time()
    while (0 < sum(network) < L^2) && (i < max_iter)
    # sum(network) de facto means number of ones/yes answers,
    # having sum(network)==0 || sum(network)==L^2 means that model converged
        i += 1
        # foreach node in the network, copy opinion from one of its neighbout
        for x in 1:L
            for y in 1:L
                decision = rand(1:4)    # randomly pick neigbour to copy opinion from
                # why doesn't Julia natively support switch-case statement huh? -__-
                if decision == 1
                    # mod1() makes sure we dont go over the index range,
                    # insted we just go back to the start
                    network[x, y] = network[mod1(x - 1, L), y] # up
                elseif decision == 2
                    network[x, y] = network[mod1(x + 1, L), y] # down
                elseif decision == 3
                    network[x, y] = network[x, mod1(y - 1, L)] # left
                else
                    network[x, y] = network[x, mod1(y + 1, L)] # right
                end
            end
        end
        push!(yesses, sum(network)) # add number of ones / yes answers to the list                  
    end
    end_time = time()

    elapsed_time = end_time - start_time

    if verbose
        # end message using fancy ternary operator
        i == max_iter ? println("Model for L=$L did not converge, max_iter=$i reached") : println("Model for L=$L converged after $i iterations")
    end

    return (i, elapsed_time, yesses)

end;

In [4]:
(iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L);

Model for L=100 converged after 17260 iterations


In [5]:
df = DataFrame(zad = 1,
               L = L,
               iter = iterations_passed,
               time = time_elapsed, 
               evolution = join(string.(ans_evolution), ";"));

# Zad 2.  
Powtarzamy dziesięciokrotnie symulacje mierząc średni czasy dojścia do konsensusu (dziesieć różnych punktów startowych). Wyznaczamy średnią z tych dziesięciu pomiarów i niepewność jej wyznaczenia.

In [6]:
for i in 1:10
    (iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L, verbose=false);
    push!(df, (2, L, iterations_passed, time_elapsed, join(string.(ans_evolution), ";")))
end;

  # Zad 3.  
Sprawdźmy czy i jak średni czas dojścia do konsensusu (z dziesięciu realizacji) zależy od rozmiaru układu $L^2$? Przyjmijmy L= 10, 50, 100, 200.

In [7]:
for L in [10, 50, 100, 200]    
    for i in 1:10
        (iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L, verbose=false);
        push!(df, (3, L, iterations_passed, time_elapsed, join(string.(ans_evolution), ";")))
    end
    println("10 x model with L=$L calculated")
end;

10 x model with L=10 calculated
10 x model with L=50 calculated
10 x model with L=100 calculated
10 x model with L=200 calculated


In [8]:
df[1:5, 1:4]

Row,zad,L,iter,time
,Int64,Int64,Int64,Float64
1,1,100,17260,1.63845
2,2,100,21178,2.08856
3,2,100,4262,0.395085
4,2,100,3648,0.338642
5,2,100,3363,0.313183


In [9]:
CSV.write("results.csv", df)

"results.csv"

In [10]:
# Krzysztof Stawarz
# EOF